# Must Run 1st and 2nd code block to define
### This program is used for only to paste 1 object class at a time

In [ ]:
# Define all parameters here

# Dict for all object classes
# folder -> class name, longest_min -> min length of image, longest_max -> max length of image
obj_dict = {
    1: {'folder': "Cpin_wNut", 'longest_min': 100, 'longest_max': 250},
    2:{'folder': "Cgoldcap", 'longest_min': 90, 'longest_max': 115},
    3: {'folder': 'Cejector', 'longest_min': 65, 'longest_max': 1000}
}

# Image resolution (X*Y)
bg_width, bg_height = 4056, 3040

# Maximum number of noise, minimum number = 1
max_num_noise = 10

## Details control for generator
# Generate image with noise (set True to enable)
noise_presence = False

# Background noise maxed-longest-side and min-longest-side
max_noise_length, min_noise_length = 120, 50

# Plain color generated background (set True to enable)
plain_color = False

# Only works when "plain_color" == True. (from black to white background, 0 to 255, put "" if wanted invisible)
shade = ""

# Overlapping degree of each instances (from 0 to 1)
overlapping_degree = 0



In [ ]:
## Parameters that should not be relevant to this piece of program

# Maximum count of object(not the noise) per image (min = 0)
max_obj_per_img = 15

# Maximum count of object per class (min = 0)
max_obj_per_class = 8

# Allowing outbounding of the instances. Instances only show portion inside the image and portion are left outside the image, set True to enable
outbound = False

# Number of synthetic image needed to generate (min = 1)
num_to_generate = 10

import cv2
import matplotlib.pyplot as plt
import os
import numpy as np
import albumentations as A
import time
import random
from tqdm import tqdm
from pathlib import Path
from datetime import datetime


### Create mask from image
def build_mask_from_img(image):
    # Make image gray before throwing into binary process
    mask = cv2.imread(image, cv2.IMREAD_UNCHANGED)

    # Binary process take place, set thresh value if needed
    thresh = 0
    mask = cv2.threshold(mask[:,:,3], thresh, 255, cv2.THRESH_BINARY)[1]

    # Bitwise the image with NOT operator to invert it
    mask = cv2.bitwise_not(mask)

    if mask is None:
        mask = np.zeros_like(image, dtype=np.uint8)
    return mask


# Define and initiate the variables
PATH_MAIN = "data"

for k, v in obj_dict.items():
    # Temp var
    masks_path = []

    # Access into the file
    folder_name = obj_dict[k]['folder']

    # Load the path to obj image file
    files_imgs = sorted(os.listdir(os.path.join(PATH_MAIN, folder_name, 'images')))
    files_imgs = [os.path.join(PATH_MAIN, folder_name, 'images', f) for f in files_imgs]

    #create folder if folder does not exist
    check_mask_path = os.path.join(PATH_MAIN, folder_name, 'masks')
    if not os.path.exists(check_mask_path):
        os.makedirs(check_mask_path)

    # Build /masks folder from the /images folder
    for image in files_imgs:      
        # Called function to build mask
        mask = build_mask_from_img(image=image)

        ## Now we obtain a whole black coloured object with a white color background
        # Preset the output for the mask
        mask_path = Path(image).stem # Get the img path
        mask_path = f"{mask_path}.png" # Set the extension
        mask_path = os.path.join(check_mask_path, mask_path) # Define the path
        
        #save mask
        cv2.imwrite(mask_path, mask)

        #bring to outer-layer path to record
        masks_path.append(mask_path)
    obj_dict[k]['images'] = files_imgs
    obj_dict[k]['masks'] = masks_path

# Load the background files
files_bg_imgs = os.listdir(os.path.join(PATH_MAIN, 'bg'))
files_bg_imgs = [os.path.join(PATH_MAIN, 'bg', f) for f in files_bg_imgs]

# Load the path of background_noise files
files_bg_noise_imgs = os.listdir(os.path.join(PATH_MAIN, "bg_noise", "images"))
files_bg_noise_imgs = [os.path.join(PATH_MAIN, "bg_noise", "images", f) for f in files_bg_noise_imgs]

#create folder if folder does not exist
check_noise_mask_path = os.path.join(PATH_MAIN, 'bg_noise', 'masks')
if not os.path.exists(check_noise_mask_path):
    os.makedirs(check_noise_mask_path)

files_bg_noise_masks = []

#Build mask for noise
for image in files_bg_noise_imgs:
    #Build mask 
    noise_mask = build_mask_from_img(image = image)

    ## Now we obtain a whole black coloured object with a white color background
    # Preset the output for the mask
    noise_mask_path = Path(image).stem # Get the img path
    noise_mask_path = f"{noise_mask_path}.png" # Set the extension
    noise_mask_path = os.path.join(check_noise_mask_path, noise_mask_path) # Define the path
    
    #save mask
    cv2.imwrite(noise_mask_path, noise_mask)

    #bring to outer-layer path to record
    files_bg_noise_masks.append(noise_mask_path)


### Gets the image and corresponding mask
def get_img_and_mask(img_path, mask_path):

    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    mask = cv2.imread(mask_path)
    mask = cv2.cvtColor(mask, cv2.COLOR_BGR2RGB)
    
    mask_b = mask[:,:,0] == 0 # This is boolean mask
    mask = mask_b.astype(np.uint8) # This is binary mask
    
    return img, mask


### Resize the image
def resize_img(img, desired_max, desired_min=None):
   
    h, w = img.shape[0], img.shape[1]
    
    longest, shortest = max(h, w), min(h, w)
    longest_new = desired_max
    if desired_min:
        shortest_new = desired_min
    else:
        shortest_new = int(shortest * (longest_new / longest))
    
    if h > w:
        h_new, w_new = longest_new, shortest_new
    else:
        h_new, w_new = shortest_new, longest_new
        
    transform_resize = A.Compose([
        A.Sequential([
        A.Resize(h_new, w_new, interpolation=1, always_apply=False, p=1)
        ], p=1)
    ])

    transformed = transform_resize(image=img)
    img_r = transformed["image"]
        
    return img_r


### Resize + transforming the object
def resize_transform_obj(img, mask, longest_min, longest_max, transforms=False):
   
    h, w = mask.shape[0], mask.shape[1]
    
    longest, shortest = max(h, w), min(h, w)
    longest_new = np.random.randint(longest_min, longest_max)
    shortest_new = int(shortest * (longest_new / longest))
    
    if h > w:
        h_new, w_new = longest_new, shortest_new
    else:
        h_new, w_new = shortest_new, longest_new
        
    transform_resize = A.Resize(h_new, w_new, interpolation=1, always_apply=False, p=1)

    transformed_resized = transform_resize(image=img, mask=mask)
    img_t = transformed_resized["image"]
    mask_t = transformed_resized["mask"]
        
    if transforms:
        transformed = transforms(image=img_t, mask=mask_t)
        img_t = transformed["image"]
        mask_t = transformed["mask"]
        
    return img_t, mask_t

transforms_bg_obj = A.Compose([
    A.RandomRotate90(p=1),
    A.ColorJitter(brightness=0.3,
                  contrast=0.3,
                  saturation=0.3,
                  hue=0.07,
                  always_apply=False,
                  p=1),
    A.Blur(blur_limit=(3,7),
           always_apply=False,
           p=0.5)
])

transforms_obj = A.Compose([
    A.RandomRotate90(p=1),
    A.RandomBrightnessContrast(brightness_limit=(-0.1, 0.2),
                               contrast_limit=0.1,
                               brightness_by_max=True,
                               always_apply=False,
                               p=1)
])


### Add object into background
def add_obj(img_comp, mask_comp, img, mask, x, y, idx):
    '''
    img_comp - composition of objects
    mask_comp - composition of objects` masks
    img - image of object
    mask - binary mask of object
    x, y - coordinates where center of img is placed
    Function returns img_comp in CV2 RGB format + mask_comp
    '''
    h_comp, w_comp = img_comp.shape[0], img_comp.shape[1]
    
    h, w = img.shape[0], img.shape[1]
    
    x = x - int(w/2) # Find the left side of img
    y = y - int(h/2) # Find the up side of img
    # So that (x, y) now being upper left corner of img
    
    # Cast into int to ensure Dataframe can manage
    x = int(x)
    y = int(y)

    mask_b = mask == 1
    mask_rgb_b = np.stack([mask_b, mask_b, mask_b], axis=2)


    # h_part - part of the image which gets into the frame of img_comp along y-axis
    # w_part - part of the image which gets into the frame of img_comp along x-axis
    if x >= 0 and y >= 0:
    
        h_part = int(h - max(0, y+h-h_comp)) 
        w_part = int(w - max(0, x+w-w_comp)) 

        # Assign part of img_comp by new value. '~' bitwise NOT. 
        img_comp[y:y+h_part, x:x+w_part, :] = img_comp[y:y+h_part, x:x+w_part, :] * ~mask_rgb_b[0:h_part, 0:w_part, :] + (img * mask_rgb_b)[0:h_part, 0:w_part, :]
        mask_comp[y:y+h_part, x:x+w_part] = mask_comp[y:y+h_part, x:x+w_part] * ~mask_b[0:h_part, 0:w_part] + (idx * mask_b)[0:h_part, 0:w_part]
        mask_added = mask[0:h_part, 0:w_part]
        
    elif x < 0 and y < 0:
        
        h_part = int(h + y)
        w_part = int(w + x)
        
        img_comp[0:0+h_part, 0:0+w_part, :] = img_comp[0:0+h_part, 0:0+w_part, :] * ~mask_rgb_b[h-h_part:h, w-w_part:w, :] + (img * mask_rgb_b)[h-h_part:h, w-w_part:w, :]
        mask_comp[0:0+h_part, 0:0+w_part] = mask_comp[0:0+h_part, 0:0+w_part] * ~mask_b[h-h_part:h, w-w_part:w] + (idx * mask_b)[h-h_part:h, w-w_part:w]
        mask_added = mask[h-h_part:h, w-w_part:w]
        
    elif x < 0 and y >= 0:
        
        h_part = int(h - max(0, y+h-h_comp))
        w_part = int(w + x)
        
        img_comp[y:y+h_part, 0:0+w_part, :] = img_comp[y:y+h_part, 0:0+w_part, :] * ~mask_rgb_b[0:h_part, w-w_part:w, :] + (img * mask_rgb_b)[0:h_part, w-w_part:w, :]
        mask_comp[y:y+h_part, 0:0+w_part] = mask_comp[y:y+h_part, 0:0+w_part] * ~mask_b[0:h_part, w-w_part:w] + (idx * mask_b)[0:h_part, w-w_part:w]
        mask_added = mask[0:h_part, w-w_part:w]
        
    elif x >= 0 and y < 0:
        
        h_part = int(h + y)
        w_part = int(w - max(0, x+w-w_comp))
        
        img_comp[0:0+h_part, x:x+w_part, :] = img_comp[0:0+h_part, x:x+w_part, :] * ~mask_rgb_b[h-h_part:h, 0:w_part, :] + (img * mask_rgb_b)[h-h_part:h, 0:w_part, :]
        mask_comp[0:0+h_part, x:x+w_part] = mask_comp[0:0+h_part, x:x+w_part] * ~mask_b[h-h_part:h, 0:w_part] + (idx * mask_b)[h-h_part:h, 0:w_part]
        mask_added = mask[h-h_part:h, 0:w_part]
    
    return img_comp, mask_comp, mask_added


### Add noise to background
def create_bg_with_noise(files_bg_imgs,
                         files_bg_noise_imgs,
                         files_bg_noise_masks,
                         bg_max=bg_width, # Image max side length in pixel
                         bg_min=bg_height, # Image min side length in pixel
                         max_objs_to_add=max_num_noise,
                         longest_bg_noise_max=120,
                         longest_bg_noise_min=50,
                         blank_bg=plain_color):
    
    if blank_bg:
        img_comp_bg = np.ones((bg_min, bg_max,3), dtype=np.uint8) * 255
        mask_comp_bg = np.zeros((bg_min, bg_max), dtype=np.uint8)
    else:    
        idx = np.random.randint(len(files_bg_imgs))
        img_bg = cv2.imread(files_bg_imgs[idx])
        img_bg = cv2.cvtColor(img_bg, cv2.COLOR_BGR2RGB)
        img_comp_bg = resize_img(img_bg, bg_max, bg_min)
        mask_comp_bg = np.zeros((img_comp_bg.shape[0], img_comp_bg.shape[1]), dtype=np.uint8)

    # Randomize obj and randomize size number
    for i in range(1, np.random.randint(max_objs_to_add) + 2):
        idx = np.random.randint(len(files_bg_noise_imgs))
        img, mask = get_img_and_mask(files_bg_noise_imgs[idx], files_bg_noise_masks[idx])
        x, y = np.random.randint(img_comp_bg.shape[1]), np.random.randint(img_comp_bg.shape[0])
        img_t, mask_t = resize_transform_obj(img, mask, longest_bg_noise_min, longest_bg_noise_max, transforms=transforms_bg_obj)
        img_comp_bg, _, _ = add_obj(img_comp_bg, mask_comp_bg, img_t, mask_t, x, y, i)
        
    return img_comp_bg


### Degree of overlapping control
def check_areas(mask_comp, obj_areas, overlap_degree=overlapping_degree):
    obj_ids = np.unique(mask_comp).astype(np.uint8)[1:-1]
    masks = mask_comp == obj_ids[:, None, None]
    
    ok = True
    
    if len(np.unique(mask_comp)) != np.max(mask_comp) + 1:
        return False
    
    for idx, mask in enumerate(masks):
        if np.count_nonzero(mask) / obj_areas[idx] < 1 - overlap_degree:
            ok = False
            break
            
    return ok

'''
### Creating synthetic composition
def create_composition(img_comp_bg,
                       coordinates,
                       max_objs=max_obj_per_img,
                       overlap_degree=overlapping_degree, # 0 - 1
                       max_attempts_per_obj=max_obj_per_class,
                       outbounding=outbound):

    img_comp = img_comp_bg.copy()
    h, w = img_comp.shape[0], img_comp.shape[1]
    mask_comp = np.zeros((h,w), dtype=np.uint8)
    
    obj_areas = []
    labels_comp = []
    
    # num_objs = np.random.randint(max_objs) + 2
    num_objs = max_objs + 1
    i = 1
    
    for _ in range(1, num_objs):
        
        # Access corresponding key value in obj_dict
        obj_idx = np.random.randint(len(obj_dict)) + 1
        
        for _ in range(max_attempts_per_obj):
            # Get sample number and determine which instances to use
            imgs_number = len(obj_dict[obj_idx]['images'])
            idx = np.random.randint(imgs_number)
            img_path = obj_dict[obj_idx]['images'][idx]
            mask_path = obj_dict[obj_idx]['masks'][idx]
            img, mask = get_img_and_mask(img_path, mask_path)

            longest_min = obj_dict[obj_idx]['longest_min']
            longest_max = obj_dict[obj_idx]['longest_max']
            img, mask = resize_transform_obj(img,
                                             mask,
                                             longest_min,
                                             longest_max,
                                             transforms=transforms_obj)

            # Make changes to x, y if wanted to define the size of instances composite into the image
            # Needed to modify that the random number is always yStart = imgh/2 and xStart = imgw/2 at least and always between imgyMax-h/2 and imgxMax-w/2
            # If outbounding == True, means instances added are allowed to clipped in half of the images

            # instance_h, instance_w = img.shape[0], img.shape[1]
            # x, y = 0, 0
            
            # if outbounding:
            #     x, y = np.random.randint(w), np.random.randint(h)
            # else:
            #     x, y = ((instance_w/2)+np.random.randint((w-instance_w))), ((instance_h/2)+np.random.randint((h-instance_h)))
            x, y = coordinates[i-1][0],coordinates[i-1][1]

            if i == 1:
                img_comp, mask_comp, mask_added = add_obj(img_comp,
                                                          mask_comp,
                                                          img,
                                                          mask,
                                                          x,
                                                          y,
                                                          i)
                obj_areas.append(np.count_nonzero(mask_added))
                labels_comp.append(obj_idx)
                i += 1
                break
            else:        
                img_comp_prev, mask_comp_prev = img_comp.copy(), mask_comp.copy()
                img_comp, mask_comp, mask_added = add_obj(img_comp,
                                                          mask_comp,
                                                          img,
                                                          mask,
                                                          x,
                                                          y,
                                                          i)
                ok = check_areas(mask_comp, obj_areas, overlap_degree)
                if ok:
                    obj_areas.append(np.count_nonzero(mask_added))
                    labels_comp.append(obj_idx)
                    i += 1
                    break
                else:
                    img_comp, mask_comp = img_comp_prev.copy(), mask_comp_prev.copy()        
        
    return img_comp, mask_comp, labels_comp, obj_areas
'''

# Create synthetic composition
def create_composition(img_comp_bg,
                       obj_range,
                       coordinates_dict,
                       overlap_degree=overlapping_degree,
                       outbounding=outbound):
    img_comp = img_comp_bg.copy()
    h, w = img_comp.shape[0], img_comp.shape[1]
    mask_comp = np.zeros((h,w),dtype=np.int8)

    obj_areas = []
    labels_comp = []

    i = 1
    for key, val in obj_dict.items():
        # Check if coordinates_dict needed the specific class object, else next obj class
        if val['folder'] not in coordinates_dict: # Get specific class and coordinates
            continue # If coordinates absence, skip current class
        if val['folder'] in obj_range: # If folder not in obj_range, it'll randomly select the instances from corresponding class
            if obj_range[val['folder']]: # Something inside list
                temp_list_file = []
                temp_list_mask = []
                for filename in obj_range[val['folder']]:
                    file = os.path.join("data", val['folder'], "images", filename)
                    if (os.path.exists(file)):
                        temp_list_file.append(file)
                        temp_mask = os.path.join("data", val['folder'], "masks", filename)
                        temp_list_mask.append(temp_mask)
                if temp_list_file:
                    obj_dict[key]['images'] = temp_list_file
                    obj_dict[key]['masks']  = temp_list_mask

        # Retrieve number of sample
        imgs_number = len(obj_dict[key]['images'])
        
        ## If class object needed
        # Obtain current class obj coordinates from coordinates_dict
        for coordinate in coordinates_dict[val['folder']]:
            # Determine which instance to be use, get img path and mask path
            idx = np.random.randint(imgs_number)
            img_path = obj_dict[key]['images'][idx]
            mask_path = obj_dict[key]['masks'][idx]
            img, mask = get_img_and_mask(img_path, mask_path)

            # Decide transformation for the selected instances
            longest_min = obj_dict[key]['longest_min']
            longest_max = obj_dict[key]['longest_max']
            img, mask = resize_transform_obj(img,
                                             mask,
                                             longest_min,
                                             longest_max,
                                             transforms=transforms_obj)

            # Add obj with specific classes into bg by coordinate
            img_comp_prev, mask_comp_prev = img_comp.copy(), mask_comp.copy()
            img_comp, mask_comp, mask_added = add_obj(img_comp,
                                                      mask_comp,
                                                      img,
                                                      mask,
                                                      coordinate[0],
                                                      coordinate[1],
                                                      i)
            ok = True
            if i > 1:
                ok = check_areas(mask_comp, obj_areas, overlap_degree)
            if ok:
                obj_areas.append(np.count_nonzero(mask_added))
                labels_comp.append(key)
                i += 1
            else:
                img_comp, mask_comp = img_comp_prev.copy(), mask_comp_prev.copy()
    return img_comp, mask_comp, labels_comp, obj_areas

### Create Yolo annotations
def create_yolo_annotations(mask_comp, labels_comp):
    comp_w, comp_h = mask_comp.shape[1], mask_comp.shape[0]
    
    obj_ids = np.unique(mask_comp).astype(np.uint8)[1:]
    masks = mask_comp == obj_ids[:, None, None]
    annotations_yolo = []
    for i in range(len(labels_comp)):
        pos = np.where(masks[i])
        xmin = np.min(pos[1])
        xmax = np.max(pos[1])
        ymin = np.min(pos[0])
        ymax = np.max(pos[0])

        xc = (xmin + xmax) / 2
        yc = (ymin + ymax) / 2
        w = xmax - xmin
        h = ymax - ymin

        annotations_yolo.append([labels_comp[i] - 1,
                                 round(xc/comp_w, 5),
                                 round(yc/comp_h, 5),
                                 round(w/comp_w, 5),
                                 round(h/comp_h, 5)])

    return annotations_yolo

def get_bg(img_bg = ""):
    if (img_bg != "") and (os.path.exists(os.path.join("data","bg",img_bg))):
        img_bg = cv2.imread(os.path.join("data","bg",img_bg))
    elif((files_bg_imgs) and (plain_color == False)):
        img_bg = cv2.imread(random.choice(files_bg_imgs))
    else:
        img_comp_bg = np.ones((bg_height, bg_width, 3), dtype=np.uint8)*255
        if (isinstance(shade, int)):
            img_comp_bg.fill(shade)
        mask_comp_bg = np.zeros((bg_height,bg_width), dtype=np.uint8)
        return img_comp_bg, mask_comp_bg
    img_bg = cv2.cvtColor(img_bg, cv2.COLOR_BGR2RGB)
    img_comp_bg = resize_img(img_bg, bg_width, bg_height)
    mask_comp_bg = np.zeros((img_comp_bg.shape[0], img_comp_bg.shape[1]), dtype=np.uint8)
    return img_comp_bg, mask_comp_bg

# Load and pick a random bg image

In [ ]:
## Put file name and extension at img_bg. img_bg = "example.png" 
img_bg = ""
img_comp_bg = ""
mask_comp_bg = ""
# if ((files_bg_imgs) and (plain_color == False)):
#     img_bg = cv2.imread(random.choice(files_bg_imgs))
#     img_bg = cv2.cvtColor(img_bg, cv2.COLOR_BGR2RGB)
#     img_comp_bg = resize_img(img_bg, bg_width, bg_height)
#     mask_comp_bg = np.zeros((img_comp_bg.shape[0], img_comp_bg.shape[1]), dtype=np.uint8)
# elif (plain_color == True):
#     img_comp_bg = np.ones((bg_height, bg_width, 3), dtype=np.uint8)*255
#     if (isinstance(shade, int)):
#         img_comp_bg.fill(shade)
#     mask_comp_bg = np.zeros((bg_height,bg_width), dtype=np.uint8)

img_comp_bg, mask_comp_bg = get_bg(img_bg)

# Add random noise into image if noise_presence == True
if noise_presence:
    for i in range(1, np.random.randint(max_num_noise) + 2):
        idx = np.random.randint(len(files_bg_noise_imgs))
        img, mask = get_img_and_mask(files_bg_noise_imgs[idx], files_bg_noise_masks[idx])
        x, y = np.random.randint(img_comp_bg.shape[1]), np.random.randint(img_comp_bg.shape[0])
        img_t, mask_t = resize_transform_obj(img, mask, min_noise_length,max_noise_length, transforms=transforms_bg_obj)
        img_comp_bg, _, _ = add_obj(img_comp_bg, mask_comp_bg, img_t, mask_t, x, y, i)

## View current generated background
# plt.imshow(img_comp_bg)

# Key in the pixel to be use as coordinates for instances

In [ ]:
## Write all the coordinate(X,Y) of the center of instances that want to be pasted
# [X,Y]
coordinates_dict = {
    'Cpin_wNut':[[300,400], [1800,400], [2000,1600],[1500,2900]],
    'Cgoldcap' :[]
}

# Range of instances, can select corresponding instances by name of file. "Cpin_wNut_1.png". 
# Will neglect if the filename provided is not exist.
# If want to select all, can leave the list empty
obj_range = {
    'Cpin_wNut':["Cpin_wNut_1.png","Cpin_wNut_2.png"],
    'Cgoldcap' :[]
}
# Make a copy
img_bg = img_comp_bg.copy()

# File path ensure to be exist
os.makedirs(os.path.join("dataset","train","images"), exist_ok=True)
os.makedirs(os.path.join("dataset","train","labels"), exist_ok=True)

# Get current data and time
time.sleep(1)
clock = datetime.today().strftime('%Y%m%d%H%M%S')

img_comp, mask_comp, labels_comp, _ = create_composition(img_comp_bg=img_bg,
                                                         obj_range=obj_range,
                                                         coordinates_dict=coordinates_dict,
                                                         overlap_degree=overlapping_degree)

fig, ax = plt.subplots(1, 2, figsize=(16, 7))
ax[0].imshow(img_comp)
ax[0].set_title('Composition', fontsize=18)
ax[1].imshow(mask_comp)
ax[1].set_title('Composition mask', fontsize=18)

## Save the image into dataset

In [ ]:
img_comp = cv2.cvtColor(img_comp, cv2.COLOR_RGB2BGR)
img_name = '{}_synthetic.png'.format(clock)
txt_name = '{}_synthetic.txt'.format(clock)
cv2.imwrite(os.path.join("dataset","train","images",img_name),img_comp)

annotations_yolo = create_yolo_annotations(mask_comp, labels_comp)

for j in range(len(annotations_yolo)):
    with open(os.path.join("dataset","train","labels",txt_name),"a") as f:
        f.write(' '.join(str(el) for el in annotations_yolo[j]) + "\n")

## Store a txt contains class info
# Create a str storing class information
temp_str = ''
for key, value in coordinates_dict.items():
    instance_name = key
    temp_str += instance_name + '\n'

# Write class infomation into a txt file at /dataset
temp_path = os.path.join(os.getcwd(),'dataset','Classes.txt')
info = open(temp_path, 'w')
info.write(temp_str)
info.close()
